In [38]:
import pandas as pd
from functools import reduce
import os
import pandas as pd
import glob
import json

## Read File

In [39]:
province_dict = {}
files = glob.glob("data/raw/*.xlsx")
files

['data/raw\\doctors.xlsx',
 'data/raw\\elderly.xlsx',
 'data/raw\\equipment.xlsx',
 'data/raw\\insurance.xlsx',
 'data/raw\\opd_ipd.xlsx']

In [40]:
doctor = pd.read_excel('data/raw\\doctors.xlsx')

In [41]:
# คอลัมน์วิชาชีพหลัก
main_cols = ["แพทย์", "ทันตแพทย์", "เภสัชกร", "พยบ.วิชาชีพ"]

# คอลัมน์เฉพาะทาง = คอลัมน์ทั้งหมด ยกเว้น 5 ตัวหลัก (รวมจังหวัด)
specialty_cols = [col for col in doctor.columns if col not in main_cols + ["จังหวัด"]]

# รวมเฉพาะทางทั้งหมดเป็น 1 คอลัมน์
doctor_final = pd.DataFrame()
doctor_final["จังหวัด"] = doctor["จังหวัด"]

# 4 วิชาชีพหลัก
for col in main_cols:
    doctor_final[col] = doctor[col]

# รวมเฉพาะทางทั้งหมด
doctor_final["บุคลากรเฉพาะทางรวม"] = doctor[specialty_cols].sum(axis=1)

doctor_final.head()

,จังหวัด,แพทย์,ทันตแพทย์,เภสัชกร,พยบ.วิชาชีพ,บุคลากรเฉพาะทางรวม
0,กรุงเทพมหานคร,12537,1288,3437,41766,14822
1,กระบี่,194,58,94,1081,586
2,กาญจนบุรี,296,88,142,1883,902
3,กาฬสินธุ์,288,81,175,1975,874
4,กำแพงเพชร,204,68,115,1320,868


In [34]:
doctor_final.to_excel("data/raw/doctors2.xlsx",index=False)

## Create key

In [46]:
dict_path = "app/province_dict.json"
if os.path.exists(dict_path):
    with open(dict_path, "r", encoding="utf-8") as f:
        province_dict = json.load(f)
else:
    province_dict = {}

province_dict

{'กรุงเทพมหานคร': 'P1000',
 'กระบี่': 'P1001',
 'กาญจนบุรี': 'P1002',
 'กาฬสินธุ์': 'P1003',
 'กำแพงเพชร': 'P1004',
 'ขอนแก่น': 'P1005',
 'จันทบุรี': 'P1006',
 'ฉะเชิงเทรา': 'P1007',
 'ชลบุรี': 'P1008',
 'ชัยนาท': 'P1009',
 'ชัยภูมิ': 'P1010',
 'ชุมพร': 'P1011',
 'เชียงราย': 'P1012',
 'เชียงใหม่': 'P1013',
 'ตรัง': 'P1014',
 'ตราด': 'P1015',
 'ตาก': 'P1016',
 'นครนายก': 'P1017',
 'นครปฐม': 'P1018',
 'นครพนม': 'P1019',
 'นครราชสีมา': 'P1020',
 'นครศรีธรรมราช': 'P1021',
 'นครสวรรค์': 'P1022',
 'นนทบุรี': 'P1023',
 'นราธิวาส': 'P1024',
 'น่าน': 'P1025',
 'บึงกาฬ': 'P1026',
 'บุรีรัมย์': 'P1027',
 'ปทุมธานี': 'P1028',
 'ประจวบคีรีขันธ์': 'P1029',
 'ปราจีนบุรี': 'P1030',
 'ปัตตานี': 'P1031',
 'พระนครศรีอยุธยา': 'P1032',
 'พะเยา': 'P1033',
 'พังงา': 'P1034',
 'พัทลุง': 'P1035',
 'พิจิตร': 'P1036',
 'พิษณุโลก': 'P1037',
 'เพชรบุรี': 'P1038',
 'เพชรบูรณ์': 'P1039',
 'แพร่': 'P1040',
 'ภูเก็ต': 'P1041',
 'มหาสารคาม': 'P1042',
 'มุกดาหาร': 'P1043',
 'แม่ฮ่องสอน': 'P1044',
 'ยะลา': 'P1045',
 'ยโส

In [ ]:
files = glob.glob("data/raw/*.xlsx")

for path in files:
    df = pd.read_excel(path)
    col = next((c for c in df.columns if "จังหวัด" in c or "Province" in c), None)
    if not col:
        continue
    for p in df[col].dropna().unique():
        if p not in province_dict:
            province_dict[p] = f"P{len(province_dict)+1000}"
            print("หาไฟล์ไม่เจอเลยสร้างใหม่")
    df["ProvinceKey"] = df[col].map(province_dict)
    df.to_excel(path.replace(".xlsx", "_with_key.xlsx"), index=False)
    print(f"{os.path.basename(path)} done")

doctors.xlsx done
elderly.xlsx done
equipment.xlsx done
insurance.xlsx done
opd_ipd.xlsx done


## Save dictionary

In [ ]:
# with open(dict_path, "w", encoding="utf-8") as f:
#     json.dump(province_dict, f, ensure_ascii=False, indent=2)
# print("province_dict.json updated")

province_dict.json updated


## Sum File

In [48]:
path = "data/raw"
files = glob.glob(os.path.join(path, "*_with_key.xlsx"))

dfs = []

for f in files:
    df = pd.read_excel(f)
    col = next((c for c in df.columns if "จังหวัด" in c or "Province" in c), None)
    if not col:
        continue
    keep_cols = [col, "ProvinceKey"]
    num_cols = df.select_dtypes(include="number").columns.tolist()
    keep_cols.extend(num_cols)
    df = df[keep_cols]
    source_name = os.path.basename(f).replace("_with_key.xlsx", "")
    df = df.rename(columns={c: f"{source_name}_{c}" for c in num_cols})
    dfs.append(df)

from functools import reduce
merged = reduce(lambda left, right: pd.merge(left, right, on=["ProvinceKey", col], how="outer"), dfs)
merged = merged.sort_values("ProvinceKey").reset_index(drop=True)

print(merged.shape)
merged.tail()

(77, 19)


,จังหวัด,ProvinceKey,doctors_แพทย์,doctors_ทันตแพทย์,doctors_เภสัชกร,doctors_พยบ.วิชาชีพ,doctors_บุคลากรเฉพาะทางรวม,elderly_จำนวนผู้สูงอายุ,equipment_เครื่องเอ็กเรย์คอมพิวเตอร์,equipment_เครื่องตรวจอวัยวะด้วยสนามแม่เหล็กไฟฟ้า,equipment_เครื่องสลายนิ่ว,equipment_เครื่องอัลตราซาวน์,equipment_เครื่องล้างไต,equipment_รถพยาบาล,equipment_จำนวนเตียง,insurance_บัตรทอง,insurance_จำนวนโรงพยาบาล,opd_ipd_จำนวนผู้ป่วยนอกเฉลี่ยต่อวัน,opd_ipd_จำนวนผู้ป่วยในเฉลี่ยต่อวัน
72,อำนาจเจริญ,P1072,95,33,71,705,467,65212,4,0,1,31,67,37,662,198,87,3030.965035,569.780822
73,อุดรธานี,P1073,588,127,312,3519,1784,288957,18,7,2,183,152,130,3235,651,234,13314.164336,2557.063014
74,อุตรดิตถ์,P1074,223,64,105,1337,669,105086,2,1,1,47,36,46,997,351,102,3310.944056,802.128767
75,อุทัยธานี,P1075,135,46,81,755,476,71108,4,0,2,32,32,50,785,156,97,3659.909091,608.221918
76,อุบลราชธานี,P1076,865,185,384,4927,2296,401029,20,5,11,139,326,162,4418,870,359,16346.776224,3863.153425


In [49]:
merged.to_excel("data/raw/Adequecy_data.xlsx",index=False)

In [ ]:
# import requests
# import pandas as pd

# url = "https://opendata-service.moph.go.th/gis/v1/getgis/provcode/12"

# # ดึงข้อมูลจาก API
# res = requests.get(url)
# data = res.json()

# # ดึงเฉพาะ features
# features = data["features"]

# # แปลงเป็น DataFrame
# df = pd.json_normalize(features)

# df


,type,geometry.coordinates,geometry.type,properties.bed,properties.distcode,properties.subdistcode,properties.address,properties.provcode,properties.postcode,properties.level_service,properties.hostype,properties.hoscode,properties.moo,properties.hosname,properties.dep
0,Feature,"[100.49238324000000000000, 13.8586378400000000...",Point,0,01,02,ถ.เรวดี,12,11000,10-1 ปฐมภูมิ,18,00993,None,โรงพยาบาลส่งเสริมสุขภาพตำบลตลาดขวัญ,21002
1,Feature,"[100.52141892000000000000, 13.8421013420000000...",Point,0,01,03,2/1,12,11000,10-1 ปฐมภูมิ,18,00995,None,โรงพยาบาลส่งเสริมสุขภาพตำบลบางเขน,21002
2,Feature,"[100.52771866140165000000, 13.8832169162396330...",Point,0,01,05,30/19 ถ.สามัคคี,12,11000,10-1 ปฐมภูมิ,18,00996,None,โรงพยาบาลส่งเสริมสุขภาพตำบลท่าทราย,21002
3,Feature,"[100.52157372000000000000, 13.8724028600000000...",Point,0,01,05,ซ.ทานสัมฤทธิ์ ถ.ติวานนท์,12,11000,10-1 ปฐมภูมิ,18,00997,None,โรงพยาบาลส่งเสริมสุขภาพตำบลทานสัมฤทธิ์,21002
4,Feature,"[100.48731847000000000000, 13.8258409080000000...",Point,0,01,06,33/2 ถ.นครอินทร์,12,11000,10-1 ปฐมภูมิ,18,00998,None,โรงพยาบาลส่งเสริมสุขภาพตำบลวัดโชติการาม,21002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Feature,"[100.50016514745357000000, 13.8618321197633630...",Point,-,01,04,-,12,-,10-1 ปฐมภูมิ,13,15239,None,ศูนย์บริการสาธารณสุขที่ 4 วัดสมรโกฎิ,
83,Feature,"[100.49672583961137000000, 13.8073762337355030...",Point,-,02,02,-,12,-,10-1 ปฐมภูมิ,13,15243,None,ศูนย์บริการสาธารณสุขชุมชนหมู่บ้านสมชาย,21002
84,Feature,"[100.49725585905003000000, 13.9015229465158740...",Point,-,06,02,78/14 ม.1 ซ.สถานสงเคราะห์,12,11000,10-1 ปฐมภูมิ,08,22424,None,ศูนย์สุขภาพชุชนโรงพยาบาลชลประทาน,
85,Feature,"[100.35282806108853000000, 13.9107875291540670...",Point,-,04,04,46/69-72 ถ.บางไผ่-หนองเพรางาย,12,11110,-,16,22604,None,มิตรไมตรีคลินิกเวชกรรม (พฤกษา3),


## จำนวนโรงพยาบาล

In [1]:
import pandas as pd

In [ ]:
health = pd.read_excel('health_office.xlsx')


,ชื่อ,รหัส 9 หลักใหม่,รหัส 9 หลัก,รหัส 5 หลัก,เลขอนุญาตให้ประกอบสถานบริการสุขภาพ 11 หลัก,ประเภทองค์กร,ประเภทหน่วยบริการสุขภาพ,สังกัด,แผนก/กรม,ระดับโรงพยาบาล,...,รหัสอำเภอ,อำเภอ/เขต,รหัสตำบล,ตำบล/แขวง,หมู่,รหัสไปรษณีย์,แม่ข่าย,วันที่ก่อตั้ง,วันที่ปิดบริการ,อัพเดตล่าสุด(เริ่ม 05/09/2566)
0,คลินิกรักษ์ฟัน,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,3001.0,เมืองนครราชสีมา,300116.0,จอหอ,3,30310,NaN,NaN,NaN,31/08/2023
1,คลินิกนายแพทย์คธาวุฒิ,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,3019.0,ขามทะเลสอ,301901.0,ขามทะเลสอ,3,30280,NaN,NaN,NaN,06/10/2023
2,คลินิกนายแพทย์เรืองศักดิ์,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,3002.0,ครบุรี,300201.0,แชะ,4,30250,NaN,NaN,NaN,31/08/2023
3,คลินิกทีปวิทย์การแพทย์,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,3002.0,ครบุรี,300208.0,บ้านใหม่,9,30250,NaN,NaN,NaN,06/10/2023
4,คลินิกแพทย์นิพนธ์,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,3009.0,โนนไทย,300901.0,โนนไทย,0,30220,NaN,NaN,NaN,09/10/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43398,ธนภัทรคลินิกเวชกรรม,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,2203.0,ท่าใหม่,220301.0,ท่าใหม่,NaN,22120,NaN,30/10/2023,NaN,03/10/2025
43399,บวรพรรณคลินิการแพทย์แผนไทย (การแพทย์แผนไทยประย...,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,6601.0,เมืองพิจิตร,660101.0,ในเมือง,-,66000,NaN,11/10/2015,NaN,03/10/2025
43400,ณัฐชญาน์คลินิกการพยาบาลและการผดุงครรภ์,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,3011.0,ขามสะแกแสง,301106.0,หนองหัวฟาน,8,30290,NaN,04/09/2025,NaN,03/10/2025
43401,คลินิกเวชกรรมกรุงเทพระยอง สาขานิคมพัฒนา ซอย 8,NaN,NaN,NaN,NaN,เอกชน,คลินิกเอกชน,เอกชน,NaN,NaN,...,2108.0,นิคมพัฒนา,210803.0,พนานิคม,2,21180,NaN,28/08/2025,NaN,03/10/2025


In [25]:
keep = ['โรงพยาบาลชุมชน','โรงพยาบาลส่งเสริมสุขภาพตำบล','โรงพยาบาลเอกชน','โรงพยาบาล นอก สธ.','โรงพยาบาล/ศูนย์บริการสาธารณสุข สาขา','โรงพยาบาลทั่วไป','โรงพยาบาล นอก สป.สธ.','โรงพยาบาลศูนย์']

filtered = health[health['ประเภทหน่วยบริการสุขภาพ'].isin(keep)]
filtered

,ชื่อ,รหัส 9 หลักใหม่,รหัส 9 หลัก,รหัส 5 หลัก,เลขอนุญาตให้ประกอบสถานบริการสุขภาพ 11 หลัก,ประเภทองค์กร,ประเภทหน่วยบริการสุขภาพ,สังกัด,แผนก/กรม,ระดับโรงพยาบาล,...,รหัสอำเภอ,อำเภอ/เขต,รหัสตำบล,ตำบล/แขวง,หมู่,รหัสไปรษณีย์,แม่ข่าย,วันที่ก่อตั้ง,วันที่ปิดบริการ,อัพเดตล่าสุด(เริ่ม 05/09/2566)
13,โรงพยาบาลทั่วไปขนาดเล็กรัตนาธิเบศร์,NaN,NaN,NaN,NaN,เอกชน,โรงพยาบาลเอกชน,เอกชน,-,NaN,...,1201.0,เมืองนนทบุรี,120104.0,บางกระสอ,0,11000,NaN,NaN,NaN,31/08/2023
14,โรงพยาบาลตุลาการเฉลิมพระเกียรติ,NaN,NaN,NaN,NaN,รัฐบาล,โรงพยาบาล นอก สธ.,สำนักงานศาลยุติธรรม,-,NaN,...,7307.0,พุทธมณฑล,730702.0,คลองโยง,2,73170,NaN,28/01/1952,NaN,31/08/2023
18,ศูนย์บริการสาธารณสุขแห่งที่ 2 เทศบาลเมืองบางศร...,NaN,NaN,NaN,NaN,รัฐบาล,โรงพยาบาล/ศูนย์บริการสาธารณสุข สาขา,องค์กรปกครองส่วนท้องถิ่น,NaN,NaN,...,1201.0,เมืองนนทบุรี,120107.0,บางศรีเมือง,4,11000,เทศบาลตำบลบางศรีเมือง,01/08/2020,NaN,30/01/2024
162,โรงพยาบาลส่งเสริมสุขภาพตำบลนิบงบารู,NaN,NaN,NaN,NaN,รัฐบาล,โรงพยาบาลส่งเสริมสุขภาพตำบล,กระทรวงสาธารณสุข,สำนักงานปลัดกระทรวงสาธารณสุข,NaN,...,9501.0,เมืองยะลา,950116.0,สะเตงนอก,7,95000,โรงพยาบาลส่งเสริมสุขภาพตำบลสะเตงนอก ตำบลสะเตงนอก,03/09/2013,NaN,22/01/2025
232,โรงพยาบาลส่งเสริมสุขภาพตำบลบ้านดอนธรรม ตำบลใหม...,NaN,NaN,NaN,NaN,รัฐบาล,โรงพยาบาลส่งเสริมสุขภาพตำบล,กระทรวงสาธารณสุข,สำนักงานปลัดกระทรวงสาธารณสุข,NaN,...,5203.0,เกาะคา,520309.0,ใหม่พัฒนา,3,52130,NaN,NaN,NaN,31/08/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42668,โรงพยาบาลทั่วไปขนาดใหญ่จุฬารัตน์ 3 อินเตอร์,NaN,NaN,NaN,NaN,เอกชน,โรงพยาบาลเอกชน,เอกชน,NaN,NaN,...,1103.0,บางพลี,110303.0,บางปลา,11,10540,NaN,30/04/2018,NaN,29/07/2025
42859,โรงพยาบาลกรุงไทย นอร์ทเทิร์น โรงพยาบาลทั่วไปขน...,NaN,NaN,NaN,NaN,เอกชน,โรงพยาบาลเอกชน,เอกชน,NaN,NaN,...,1305.0,ลาดหลุมแก้ว,130503.0,คูบางหลวง,9,12140,NaN,15/08/2025,NaN,14/08/2025
42989,ศูนย์การแพทย์แผนไทยและการแพทย์ทางเลือก เขตสุขภ...,NaN,NaN,NaN,NaN,รัฐบาล,โรงพยาบาล/ศูนย์บริการสาธารณสุข สาขา,กระทรวงสาธารณสุข,กรมการแพทย์แผนไทยและการแพทย์ทางเลือก,NaN,...,4024.0,บ้านแฮด,402403.0,โนนสมบูรณ์,11,40110,โรงพยาบาลการแพทย์แผนไทยและการแพทย์ผสมผสาน,09/09/2025,NaN,15/09/2025
43132,ศูนย์การแพทย์แผนไทยและการแพทย์ทางเลือก สาขากรม...,NaN,NaN,NaN,NaN,รัฐบาล,โรงพยาบาล/ศูนย์บริการสาธารณสุข สาขา,กระทรวงสาธารณสุข,กรมการแพทย์แผนไทยและการแพทย์ทางเลือก,NaN,...,1201.0,เมืองนนทบุรี,120101.0,สวนใหญ่,NaN,11000,โรงพยาบาลการแพทย์แผนไทยและการแพทย์ผสมผสาน,01/10/2025,NaN,05/09/2025


In [26]:
filtered.groupby('จังหวัด')['ประเภทหน่วยบริการสุขภาพ'].count()

จังหวัด
กระบี่            40
กรุงเทพมหานคร    231
กาญจนบุรี         23
กาฬสินธุ์        125
กำแพงเพชร         51
                ... 
เพชรบุรี         124
เพชรบูรณ์        111
เลย              113
แพร่              13
แม่ฮ่องสอน        73
Name: ประเภทหน่วยบริการสุขภาพ, Length: 77, dtype: int64

In [27]:
import pandas as pd

# Group by จังหวัด แล้วนับจำนวน
df_count = (
    filtered.groupby('จังหวัด')['ประเภทหน่วยบริการสุขภาพ']
    .count()
    .reset_index()
    .rename(columns={'ประเภทหน่วยบริการสุขภาพ': 'จำนวนหน่วยบริการสุขภาพ'})
)

# บันทึกเป็น Excel
df_count.to_excel('count_by_province.xlsx', index=False)

print("บันทึกไฟล์เรียบร้อย: count_by_province.xlsx")


บันทึกไฟล์เรียบร้อย: count_by_province.xlsx


## Elderly

In [1]:
import pandas as pd

In [ ]:
dfs = []
for year in range(2557, 2569):
    df = globals()[f"x{year}"].copy()
    cols = [f"m{i}" for i in range(60, 103)] + [f"f{i}" for i in range(60, 103)]
    df["elderly_total"] = df[cols].sum(axis=1)
    df["year"] = year
    dfs.append(df[["year", "a_name", "elderly_total"]])

# รวมทุกปีเป็นตารางเดียว
combined = pd.concat(dfs, ignore_index=True)

# Pivot ให้จังหวัดเป็นคอลัมน์ แถวเป็นปี
pivoted = combined.pivot(index="year", columns="a_name", values="elderly_total")
pivoted

a_name,กระบี่,กรุงเทพมหานคร,กาญจนบุรี,กาฬสินธุ์,กำแพงเพชร,ขอนแก่น,จันทบุรี,ฉะเชิงเทรา,ชลบุรี,ชัยนาท,...,อุทัยธานี,อุบลราชธานี,อ่างทอง,เชียงราย,เชียงใหม่,เพชรบุรี,เพชรบูรณ์,เลย,แพร่,แม่ฮ่องสอน
year,,,,,,,,,,,,,,,,,,,,,
2557,44177,827213,110683,126436,105284,250501,77382,101208,160236,62457,...,54589,227014,51900,174233,256332,74876,146636,88794,79300,29212
2558,46386,866972,116295,131836,110197,261297,80616,104988,168704,64730,...,56560,236209,53519,183767,270708,77851,153353,92577,82927,30429
2559,48139,905237,120680,137344,114416,271902,83455,108234,176541,66455,...,58637,246166,55047,198329,285799,80308,158803,97240,86845,32158
2560,49714,932228,124595,141963,117697,280851,85732,110687,182696,67793,...,60105,254623,56002,208784,299026,82676,162081,101063,90391,33279
2561,52014,895714,130593,147902,122474,292438,89151,114856,192196,69982,...,62231,265384,57764,220936,315872,86126,168603,105734,94373,34625
2562,54456,910441,130146,154049,127665,303622,92828,118527,199055,72018,...,64257,274528,59459,219106,322701,88753,175218,109270,98286,33059
2563,57438,969752,144076,160764,133257,318624,97175,124040,213253,74538,...,66662,288747,61528,245237,351628,93374,183708,115633,102408,37783
2564,60051,983541,146189,165062,132422,328275,100401,127535,221429,75079,...,67543,292064,62513,254275,362133,95004,185812,118046,105321,38588
2565,63164,1009696,152224,171665,137349,340855,104497,132239,232147,77631,...,69673,303720,64245,266375,377690,98672,192993,122685,109422,40139


In [ ]:
# pivoted.to_csv('Elderly/total_elderly.csv')